In [1]:
import time, datetime
from fyers_apiv3 import fyersModel
client_id = "ZHQ4IJL7TI-100"
with open("access_token", "r") as f:
    access_token = f.read()

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from indicators.syncind import SyncInd
from indicators.classic import SMA, Alligator, SmoothMA, RSI, MACD, ATR, WMA, EMA, RMA, VolumeROC, KAMA
from indicators.candle import OHLC
from indicators.trade import *

In [4]:
balance = Balance(5000)
broker = Broker(balance)

In [5]:
date_today = datetime.datetime.now().strftime("%Y-%m-%d")
date_100_p = (datetime.datetime.now() - datetime.timedelta(days=100)).strftime("%Y-%m-%d")
date_today, date_100_p

('2024-11-25', '2024-08-17')

In [6]:
response_1 = fyers.history(data={
                                "symbol": "NSE:NIFTY50-INDEX",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": date_today,
                                "range_to": date_today,
                                "cont_flag": "1"
                                })
response_1_bts = fyers.history(data={
                                "symbol": "NSE:NIFTY50-INDEX",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": date_100_p,
                                "range_to": date_today,
                                "cont_flag": "1"
                                })

len(response_1['candles']), len(response_1_bts['candles'])

(750, 25560)

In [7]:
response_5 = fyers.history(data={
                                "symbol": "NSE:NIFTY50-INDEX",
                                "resolution": "5",
                                "date_format": "1",
                                "range_from": date_today,
                                "range_to": date_today,
                                "cont_flag": "1"
                                })

response_5_bts = fyers.history(data={
                                "symbol": "NSE:NIFTY50-INDEX",
                                "resolution": "5",
                                "date_format": "1",
                                "range_from": date_100_p,
                                "range_to": date_today,
                                "cont_flag": "1"
                                })

len(response_5['candles']), len(response_5_bts['candles'])

(150, 5112)

In [8]:
date_temp = []
final_candles_list_1 = []
for i in response_1['candles']:
    if i[0] not in date_temp:
        final_candles_list_1.append(i)
        date_temp.append(i[0])
del date_temp
len(final_candles_list_1)

375

In [9]:
date_temp = []
final_candles_list_1_bts = []
for i in response_1['candles']:
    if i[0] not in date_temp:
        final_candles_list_1_bts.append(i)
        date_temp.append(i[0])
del date_temp
len(final_candles_list_1_bts)

375

In [10]:
date_temp = []
final_candles_list_5 = []
for i in response_5['candles']:
    if i[0] not in date_temp:
        final_candles_list_5.append(i)
        date_temp.append(i[0])
del date_temp
len(final_candles_list_5)

75

In [11]:
date_temp = []
final_candles_list_5_bts = []
for i in response_5['candles']:
    if i[0] not in date_temp:
        final_candles_list_5_bts.append(i)
        date_temp.append(i[0])
del date_temp
len(final_candles_list_5_bts)

75

In [12]:
candles_1 = np.array(final_candles_list_1)
candles_1.shape

(375, 6)

In [13]:
candles_1_bts = np.array(final_candles_list_1_bts)
candles_1_bts.shape

(375, 6)

In [14]:
candles_5 = np.array(final_candles_list_5)
candles_5.shape

(75, 6)

In [15]:
candles_5_bts = np.array(final_candles_list_5_bts)
candles_5_bts.shape

(75, 6)

In [16]:
sync_1 = SyncInd( 
    # SMA(5),
    # Alligator(show_jaw=False, show_teeth=False)
    KAMA(highlight=True)
)
for c in candles_1_bts:
    sync_1.append(c)
                
sync_5 = SyncInd( 
    # SMA(5),
    # Alligator(show_jaw=False, show_teeth=False)
    KAMA(highlight=True)
)
for c in candles_5_bts:
    sync_1.append(c)
sync_1.data().shape, sync_5.data().shape

((450, 8), (0,))

In [17]:
ind = 0
trend_ = 0

for c5 in candles_5:
    for _ in range(5):
        sync_1.append(candles_1[ind+_])
        # print("1m :", datetime.datetime.fromtimestamp(int(sync_1.last_candle()[0])).strftime("%D %H-%M"))
        if sync_1.data()[-2][-1] == 1 and sync_1.data()[-1][-1] and trend_ == 1:
            print("upper treand ended :", datetime.datetime.fromtimestamp(int(sync_1.last_candle()[0])).strftime("%D %H-%M"))
            trend_ = 0

    sync_5.append(c5)
    # print("\t5m :", datetime.datetime.fromtimestamp(int(sync_5.last_candle()[0])).strftime("%D %H-%M"))
    if sync_5.data()[-1][-1] == 1 and sync_5.data()[-2][-1] == -1 and trend_ == 0:
        print("upper treand started :", datetime.datetime.fromtimestamp(int(sync_5.last_candle()[0])).strftime("%D %H-%M"))
        trend_ = 1
    
    ind = ind+_+1


upper treand started : 11/25/24 10-39
upper treand ended : 11/25/24 10-47
upper treand started : 11/25/24 10-54
upper treand ended : 11/25/24 11-00
upper treand started : 11/25/24 13-25
upper treand ended : 11/25/24 13-30
upper treand started : 11/25/24 14-04
upper treand ended : 11/25/24 14-10
upper treand started : 11/25/24 14-25
upper treand ended : 11/25/24 14-29
upper treand started : 11/25/24 15-06
upper treand ended : 11/25/24 15-10


In [18]:
i

[1732528500, 24249.95, 24260.9, 24240, 24250.8, 0]